In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import numpy as np
import pandas as pd
from typing import Dict
from dataclasses import dataclass
import sqlite3
from datetime import datetime

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.monitor import Monitor

import os, sys

from datetime import datetime, timedelta
from pathlib import Path

import logging

# Configure logging
log_file = "optuna_trials4.log"  # Path to log file
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()  # For console output
    ]
)



# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from data_management.dataset_manager import DatasetManager
from trading.environments.forex_env import ForexTradingEnv

pair = "EUR_USD"
parquet_path = Path("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h") / f"{pair}.parquet"
df = pd.read_parquet(parquet_path)

dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)


@dataclass
class RewardParams:
    """Parameters controlling the reward function behavior."""
    realized_pnl_weight: float = 1.1
    unrealized_pnl_weight: float = 0.8
    holding_time_threshold: int = 7*12  # hours
    holding_penalty_factor: float = -0.00001
    max_trades_per_day: int = 6 
    overtrading_penalty_factor: float = -0.0001
    win_rate_threshold: float = 0.4
    win_rate_bonus_factor: float = 0.0005
    drawdown_penalty_factor: float = -0.0001

@dataclass
class OptimizationResult:
    """Stores results of a single trial."""
    trial_number: int
    params: Dict
    final_balance: float
    total_trades: int
    win_rate: float
    max_drawdown: float
    training_time: float

class RewardOptimizer:
    def __init__(
        self,
        train_df: pd.DataFrame,
        val_df: pd.DataFrame,
        study_name: str = "forex_reward_optimization_non_optimized_v2",
        n_timesteps: int = 500_000
    ):
        self.train_df = train_df
        self.val_df = val_df
        self.study_name = study_name
        self.n_timesteps = n_timesteps
        
        # Setup study with TPE sampler and Median pruner
        self.study = optuna.create_study(
            study_name=study_name,
            storage="sqlite:///optuna_trials.db",
            load_if_exists=True,
            sampler=TPESampler(seed=42),
            pruner=MedianPruner(
                n_startup_trials=5,
                n_warmup_steps=100_000,
                interval_steps=50_000
            ),
            direction="maximize"
        )

    def _create_env(self, df: pd.DataFrame, params: Dict, is_eval: bool = False, n_envs: int = 3) -> VecNormalize:
        """Create vectorized and normalized environment with multiple subprocesses."""
        def make_env(rank):
            def _init():
                # Wrap the environment with Monitor for logging
                env = ForexTradingEnv(
                    df=df.copy(),
                    pair='EUR_USD',
                    initial_balance=1_000_000,
                    trade_size=100_000,
                    reward_params=RewardParams(**params)
                )
                return Monitor(env)  # Wrap the environment with Monitor
            return _init

        vec_env = SubprocVecEnv([make_env(i) for i in range(n_envs)])
        env = VecNormalize(
            vec_env,
            norm_obs=True,
            norm_reward=not is_eval,
            clip_obs=10.0,
            clip_reward=10.0,
            gamma=1,
            epsilon=1e-08
        )

        return env

    def objective(self, trial: optuna.Trial) -> float:
        """Optimization objective function."""
        def get_env_attribute(env, attr_name):
            """
            Fetch an attribute from the environment, supporting SubprocVecEnv and single environments.
            Args:
                env: The environment (SubprocVecEnv, DummyVecEnv, or wrapped env).
                attr_name (str): The name of the attribute to fetch.
            Returns:
                The attribute value, or None if not found.
            """
            if hasattr(env, "envs"):  # SubprocVecEnv or DummyVecEnv
                # Get the attribute from the first environment in the vectorized stack
                try:
                    print(f"Fetching attribute {attr_name} : {env.get_attr(attr_name, indices=0)} from the first environment in the vectorized stack.")
                    return env.get_attr(attr_name, indices=0)
                except AttributeError:
                    print(f"Attribute {attr_name} : {env.get_attr(attr_name, indices=0)} not found in the first environment.")
                    # Use the latest Gymnasium recommendation: get_wrapper_attr
                    return env.get_wrapper_attr(attr_name)
            else:  # Single unwrapped environment
                print(f"Fetching attribute {attr_name} : {getattr(env.unwrapped, attr_name, None)} from the unwrapped environment.")
                return getattr(env.unwrapped, attr_name, None)
        try:
            # Sample parameters
            params = self._sample_parameters(trial)
            
            # Create environments
            train_env = self._create_env(self.train_df, params)
            eval_env = self._create_env(self.val_df, params, is_eval=True)
            
            start_time = datetime.now()
            
            # Create model
            model = PPO(
                "MultiInputPolicy",
                train_env,
                verbose=0,
                tensorboard_log=f"./tensorboard4/trial_{trial.number}"
            )

            # Setup evaluation callback
            eval_callback = EvalCallback(
                eval_env,
                best_model_save_path=f"./models/trial_{trial.number}",
                log_path=f"./logs/trial_{trial.number}",
                eval_freq=30_000,
                deterministic=True,
                render=False
            )
            
            # Train model
            model.learn(
                total_timesteps=self.n_timesteps,
                callback=eval_callback
            )
      
            try:
                save_path = f'./optuna/best_model_trial_{trial.number}/'
                os.makedirs(save_path, exist_ok=True)
                train_env.save(os.path.join(save_path, 'vecnormalize.pkl'))
                logging.info(f"Saved VecNormalize to {os.path.join(save_path, 'vecnormalize.pkl')}")
            except Exception as e:
                logging.error(f"Failed to save VecNormalize: {e}")
                raise
            training_time = (datetime.now() - start_time).total_seconds()
            
            # Get final balance from eval environment
            #! Below use oudated code, must be further changed for othjer wrappers
            final_balance = eval_env.get_attr('balance')[0]
            total_trades = eval_env.get_attr('total_trades')[0]
            win_rate = eval_env.get_attr('winning_trades')[0] / max(1, total_trades)
            #! below only works for non  ProcVec
            # final_balance = eval_env.unwrapped.balance
            # total_trades = eval_env.unwrapped.total_trades
            # win_rate = eval_env.unwrapped.winning_trades / max(1, total_trades)
            # Fetch attributes using the new utility function
            # final_balance = get_env_attribute(eval_env, 'balance')
            # total_trades = get_env_attribute(eval_env, 'total_trades')
            # winning_trades = get_env_attribute(eval_env, 'winning_trades')
            # win_rate = winning_trades / max(1, total_trades)
            # Log values
           
                
            

            # Print trial results
            print(f"\nTrial {trial.number} completed:")
            print(f"Final Balance: ${final_balance:,.2f}")
            print(f"Parameters:")
            for key, value in params.items():
                print(f"    {key}: {value}")
            print(f"Total Trades: {total_trades}")
            print(f"Win Rate: {win_rate:.2%}")
            print(f"Training Time: {training_time:.1f}s")
            print("-" * 80)
            
            return final_balance
            
        except Exception as e:
            print(f"Trial {trial.number} failed: {str(e)}")
            return float('-inf')

    def _sample_parameters(self, trial: optuna.Trial) -> Dict:
        """Sample reward parameters for trial."""
        return {
            'realized_pnl_weight': trial.suggest_float('realized_pnl_weight', 0.5, 2.0),
            'unrealized_pnl_weight': trial.suggest_float('unrealized_pnl_weight', 0.3, 1.0),
            'holding_time_threshold': trial.suggest_int('holding_time_threshold', 24, 96),
            'holding_penalty_factor': trial.suggest_float('holding_penalty_factor', -0.0001, 0.0),
            'max_trades_per_day': trial.suggest_int('max_trades_per_day', 3, 12),
            'overtrading_penalty_factor': trial.suggest_float('overtrading_penalty_factor', -0.001, 0.0),
            'win_rate_threshold': trial.suggest_float('win_rate_threshold', 0.3, 0.5),
            'win_rate_bonus_factor': trial.suggest_float('win_rate_bonus_factor', 0.0001, 0.001, log=True),
            'drawdown_penalty_factor': trial.suggest_float('drawdown_penalty_factor', -0.001, 0.0)
        }

    def optimize(self, n_trials: int = 100, n_jobs: int = 6) -> None:
        """Run optimization using Optuna's built-in parallelization."""
        self.study.optimize(
            self.objective,
            n_trials=n_trials,
            n_jobs=n_jobs,  # Number of parallel jobs
            show_progress_bar=True
        )
            
        # Print best trial after completion
        print("\nOptimization completed!")
        print("\nBest trial:")
        trial = self.study.best_trial
        print(f"Value: ${trial.value:,.2f}")
        print("Best parameters:")
        for key, value in trial.params.items():
            print(f"    {key}: {value}")


optimizer = RewardOptimizer(
    train_df=train_df,
    val_df=val_df,
    n_timesteps=90_000
)

optimizer.optimize(n_trials=30, n_jobs=2)

2024-11-22 10:24:07,399 - setting up API-client for environment practice


Dataset split sizes:
Training: 101768 samples (70.0%)
Validation: 21808 samples (15.0%)
Test: 21808 samples (15.0%)


[I 2024-11-22 10:24:07,675] Using an existing study with name 'forex_reward_optimization_non_optimized_v2' instead of creating a new one.


  0%|          | 0/30 [00:00<?, ?it/s]


Episode Summary:
Final Return: -10.92%
Total PnL: -109150.82
Total Trades: 6490
Winning Trades: 2448
Win Rate: 37.72%
Initial Balance: 1000000.00
Final Balance: 890849.18
--------------------------------------------------

Episode Summary:
Final Return: -0.62%
Total PnL: -6235.60
Total Trades: 374
Winning Trades: 133
Win Rate: 35.56%
Initial Balance: 1000000.00
Final Balance: 993764.40
--------------------------------------------------

Episode Summary:
Final Return: -0.95%
Total PnL: -9456.48
Total Trades: 1151
Winning Trades: 440
Win Rate: 38.23%
Initial Balance: 1000000.00
Final Balance: 990543.52
--------------------------------------------------

Episode Summary:
Final Return: -1.56%
Total PnL: -15561.06
Total Trades: 1646
Winning Trades: 629
Win Rate: 38.21%
Initial Balance: 1000000.00
Final Balance: 984438.94
--------------------------------------------------

Episode Summary:
Final Return: -1.56%
Total PnL: -15569.98
Total Trades: 1646
Winning Trades: 629
Win Rate: 38.21%
Init

...


Trial 42 completed:
Final Balance: $1,000,000.00
Parameters:
    realized_pnl_weight: 1.9084710882951026
    unrealized_pnl_weight: 0.48601672795687056
    holding_time_threshold: 58
    holding_penalty_factor: -3.1439374890848e-05
    max_trades_per_day: 12
    overtrading_penalty_factor: -9.769871826436643e-05
    win_rate_threshold: 0.4213364954951109
    win_rate_bonus_factor: 0.00012786159592628114
    drawdown_penalty_factor: -0.0006023009737710527
Total Trades: 0
Win Rate: 0.00%
Training Time: 155.2s
--------------------------------------------------------------------------------
[I 2024-11-22 10:39:51,651] Trial 42 finished with value: 1000000.0 and parameters: {'realized_pnl_weight': 1.9084710882951026, 'unrealized_pnl_weight': 0.48601672795687056, 'holding_time_threshold': 58, 'holding_penalty_factor': -3.1439374890848e-05, 'max_trades_per_day': 12, 'overtrading_penalty_factor': -9.769871826436643e-05, 'win_rate_threshold': 0.4213364954951109, 'win_rate_bonus_factor': 0.0001

2024-11-22 10:42:45,150 - Saved VecNormalize to ./optuna/best_model_trial_43/vecnormalize.pkl
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.balance` for environment variables or `env.get_wrapper_attr('balance')` that will search the reminding wrappers.
  logger.warn(
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.balance` for environment variables or `env.get_wrapper_attr('balance')` that will search the reminding wrappers.
  logger.warn(
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables f


Trial 43 completed:
Final Balance: $1,007,529.38
Parameters:
    realized_pnl_weight: 0.9735900045888118
    unrealized_pnl_weight: 0.5569098233671852
    holding_time_threshold: 47
    holding_penalty_factor: -2.4347354755176955e-05
    max_trades_per_day: 12
    overtrading_penalty_factor: -7.896332542263178e-05
    win_rate_threshold: 0.42047119593265053
    win_rate_bonus_factor: 0.00012364240736350166
    drawdown_penalty_factor: -0.0001725720941118327
Total Trades: 933
Win Rate: 36.12%
Training Time: 174.4s
--------------------------------------------------------------------------------
[I 2024-11-22 10:42:45,325] Trial 43 finished with value: 1007529.3829939256 and parameters: {'realized_pnl_weight': 0.9735900045888118, 'unrealized_pnl_weight': 0.5569098233671852, 'holding_time_threshold': 47, 'holding_penalty_factor': -2.4347354755176955e-05, 'max_trades_per_day': 12, 'overtrading_penalty_factor': -7.896332542263178e-05, 'win_rate_threshold': 0.42047119593265053, 'win_rate_bon

2024-11-22 10:43:25,248 - Saved VecNormalize to ./optuna/best_model_trial_44/vecnormalize.pkl
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.balance` for environment variables or `env.get_wrapper_attr('balance')` that will search the reminding wrappers.
  logger.warn(
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.balance` for environment variables or `env.get_wrapper_attr('balance')` that will search the reminding wrappers.
  logger.warn(
/Users/floriankockler/Code/anaconda_env/SB5/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.balance to get variables f


Trial 44 completed:
Final Balance: $992,505.39
Parameters:
    realized_pnl_weight: 1.702129544808096
    unrealized_pnl_weight: 0.55715102342879
    holding_time_threshold: 48
    holding_penalty_factor: -1.2704526506898781e-05
    max_trades_per_day: 11
    overtrading_penalty_factor: -0.0002553790753815415
    win_rate_threshold: 0.3680887478839023
    win_rate_bonus_factor: 0.00018751721113956945
    drawdown_penalty_factor: -0.0006095773053968038
Total Trades: 1483
Win Rate: 38.77%
Training Time: 184.0s
--------------------------------------------------------------------------------
[I 2024-11-22 10:43:25,474] Trial 44 finished with value: 992505.3888908218 and parameters: {'realized_pnl_weight': 1.702129544808096, 'unrealized_pnl_weight': 0.55715102342879, 'holding_time_threshold': 48, 'holding_penalty_factor': -1.2704526506898781e-05, 'max_trades_per_day': 11, 'overtrading_penalty_factor': -0.0002553790753815415, 'win_rate_threshold': 0.3680887478839023, 'win_rate_bonus_factor'